# Example Predictor: Linear Rollout Predictor

This example contains basic functionality for training and evaluating a linear predictor that rolls out predictions day-by-day.

First, a training data set is created from historical case and npi data.

Second, a linear model is trained to predict future cases from prior case data along with prior and future npi data.
The model is an off-the-shelf sklearn Lasso model, that uses a positive weight constraint to enforce the assumption that increased npis has a negative correlation with future cases.

Third, a sample evaluation set is created, and the predictor is applied to this evaluation set to produce prediction results in the correct format.

## Training

In [1]:
import pickle
import numpy as np
import pandas as pd
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

### Copy the data locally

In [2]:
# Main source for the training data
DATA_URL = 'https://raw.githubusercontent.com/OxCGRT/covid-policy-tracker/master/data/OxCGRT_latest.csv'
# Local file
DATA_FILE = 'data/OxCGRT_latest.csv'

In [3]:
import os
import urllib.request
if not os.path.exists('data'):
    os.mkdir('data')
urllib.request.urlretrieve(DATA_URL, DATA_FILE)

('data/OxCGRT_latest.csv', <http.client.HTTPMessage at 0x20029bcd9c8>)

In [22]:
# Load historical data from local file
df = pd.read_csv(DATA_FILE, 
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)

In [5]:
df.columns

Index(['CountryName', 'CountryCode', 'RegionName', 'RegionCode',
       'Jurisdiction', 'Date', 'C1_School closing', 'C1_Flag',
       'C2_Workplace closing', 'C2_Flag', 'C3_Cancel public events', 'C3_Flag',
       'C4_Restrictions on gatherings', 'C4_Flag', 'C5_Close public transport',
       'C5_Flag', 'C6_Stay at home requirements', 'C6_Flag',
       'C7_Restrictions on internal movement', 'C7_Flag',
       'C8_International travel controls', 'E1_Income support', 'E1_Flag',
       'E2_Debt/contract relief', 'E3_Fiscal measures',
       'E4_International support', 'H1_Public information campaigns',
       'H1_Flag', 'H2_Testing policy', 'H3_Contact tracing',
       'H4_Emergency investment in healthcare', 'H5_Investment in vaccines',
       'H6_Facial Coverings', 'H6_Flag', 'M1_Wildcard', 'ConfirmedCases',
       'ConfirmedDeaths', 'StringencyIndex', 'StringencyIndexForDisplay',
       'StringencyLegacyIndex', 'StringencyLegacyIndexForDisplay',
       'GovernmentResponseIndex', 'Gove

In [6]:
# For testing, restrict training data to that before a hypothetical predictor submission date
HYPOTHETICAL_SUBMISSION_DATE = np.datetime64("2020-07-31")
df = df[df.Date <= HYPOTHETICAL_SUBMISSION_DATE]

In [7]:
# Add RegionID column that combines CountryName and RegionName for easier manipulation of data
df['GeoID'] = df['CountryName'] + '__' + df['RegionName'].astype(str)

In [8]:
# Add new cases column
df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)

In [9]:
# Keep only columns of interest
id_cols = ['CountryName',
           'RegionName',
           'GeoID',
           'Date']
cases_col = ['NewCases']
npi_cols = ['C1_School closing',
            'C2_Workplace closing',
            'C3_Cancel public events',
            'C4_Restrictions on gatherings',
            'C5_Close public transport',
            'C6_Stay at home requirements',
            'C7_Restrictions on internal movement',
            'C8_International travel controls',
            'H1_Public information campaigns',
            'H2_Testing policy',
            'H3_Contact tracing',
            'H6_Facial Coverings']
df = df[id_cols + cases_col + npi_cols]

In [10]:
# Fill any missing case values by interpolation and setting NaNs to 0
df.update(df.groupby('GeoID').NewCases.apply(
    lambda group: group.interpolate()).fillna(0))

In [11]:
# Fill any missing NPIs by assuming they are the same as previous day
for npi_col in npi_cols:
    df.update(df.groupby('GeoID')[npi_col].ffill().fillna(0))

In [12]:
df

,CountryName,RegionName,GeoID,Date,NewCases,C1_School closing,C2_Workplace closing,C3_Cancel public events,C4_Restrictions on gatherings,C5_Close public transport,C6_Stay at home requirements,C7_Restrictions on internal movement,C8_International travel controls,H1_Public information campaigns,H2_Testing policy,H3_Contact tracing,H6_Facial Coverings
0,Aruba,NaN,Aruba__nan,2020-01-01,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Aruba,NaN,Aruba__nan,2020-01-02,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Aruba,NaN,Aruba__nan,2020-01-03,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Aruba,NaN,Aruba__nan,2020-01-04,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Aruba,NaN,Aruba__nan,2020-01-05,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
87064,Zimbabwe,NaN,Zimbabwe__nan,2020-07-27,78.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87065,Zimbabwe,NaN,Zimbabwe__nan,2020-07-28,192.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87066,Zimbabwe,NaN,Zimbabwe__nan,2020-07-29,113.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0
87067,Zimbabwe,NaN,Zimbabwe__nan,2020-07-30,62.0,3.0,1.0,2.0,3.0,1.0,2.0,2.0,4.0,2.0,1.0,1.0,4.0


In [13]:
# Set number of past days to use to make predictions
nb_lookback_days = 30

# Create training data across all countries for predicting one day ahead
X_cols = cases_col + npi_cols
y_col = cases_col
X_samples = []
y_samples = []
geo_ids = df.GeoID.unique()
for g in geo_ids:
    gdf = df[df.GeoID == g]
    all_case_data = np.array(gdf[cases_col])
    all_npi_data = np.array(gdf[npi_cols])

    # Create one sample for each day where we have enough data
    # Each sample consists of cases and npis for previous nb_lookback_days
    nb_total_days = len(gdf)
    for d in range(nb_lookback_days, nb_total_days - 1):
        X_cases = all_case_data[d-nb_lookback_days:d]

        # Take negative of npis to support positive
        # weight constraint in Lasso.
        X_npis = -all_npi_data[d - nb_lookback_days:d]

        # Flatten all input data so it fits Lasso input format.
        X_sample = np.concatenate([X_cases.flatten(),
                                   X_npis.flatten()])
        y_sample = all_case_data[d + 1]
        X_samples.append(X_sample)
        y_samples.append(y_sample)

X_samples = np.array(X_samples)
y_samples = np.array(y_samples).flatten()

In [14]:
# Helpful function to compute mae
def mae(pred, true):
    return np.mean(np.abs(pred - true))

In [15]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X_samples,
                                                    y_samples,
                                                    test_size=0.2,
                                                    random_state=301)

In [16]:
# Create and train Lasso model.
# Set positive=True to enforce assumption that cases are positively correlated
# with future cases and npis are negatively correlated.
model = Lasso(alpha=0.1,
              precompute=True,
              max_iter=10000,
              positive=True,
              selection='random')
# Fit model
model.fit(X_train, y_train)

Lasso(alpha=0.1, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=True, precompute=True, random_state=None,
      selection='random', tol=0.0001, warm_start=False)

In [17]:
# Evaluate model
train_preds = model.predict(X_train)
train_preds = np.maximum(train_preds, 0) # Don't predict negative cases
print('Train MAE:', mae(train_preds, y_train))

test_preds = model.predict(X_test)
test_preds = np.maximum(test_preds, 0) # Don't predict negative cases
print('Test MAE:', mae(test_preds, y_test))

Train MAE: 140.7115130708164
Test MAE: 152.4960998257208


In [18]:
# Inspect the learned feature coefficients for the model
# to see what features it's paying attention to.

# Give names to the features
x_col_names = []
for d in range(-nb_lookback_days, 0):
    x_col_names.append('Day ' + str(d) + ' ' + cases_col[0])
for d in range(-nb_lookback_days, 1):
    for col_name in npi_cols:
        x_col_names.append('Day ' + str(d) + ' ' + col_name)

# View non-zero coefficients
for (col, coeff) in zip(x_col_names, list(model.coef_)):
    if coeff != 0.:
        print(col, coeff)
print('Intercept', model.intercept_)

Day -7 NewCases 0.0010981690962886179
Day -6 NewCases 0.4395939174551574
Day -5 NewCases 0.21708595513072407
Day -4 NewCases 0.05899975513969943
Day -3 NewCases 0.06945280743161403
Day -2 NewCases 0.05194441075319036
Day -1 NewCases 0.2383894816999364
Day -26 C6_Stay at home requirements 4.31299480136158
Day -22 C2_Workplace closing 9.716853002891929
Day -17 C2_Workplace closing 5.766094345069354
Intercept 26.55687525188671


In [19]:
# Save model to file
if not os.path.exists('models'):
    os.mkdir('models')
with open('models/model.pkl', 'wb') as model_file:
    pickle.dump(model, model_file)

## Evaluation

Now that the predictor has been trained and saved, this section contains the functionality for evaluating it on sample evaluation data.

In [20]:
# Reload the module to get the latest changes
import predict
from importlib import reload
reload(predict)
from predict import predict_df

In [30]:
# df_eval = pd.read_csv('data/2020-09-30_historical_ip.csv')

# row_indexs = df[df['CountryName'] == 'Anguilla'].index
# row_indexs

path_to_ips_file="data/2020-09-30_historical_ip.csv"
list_countries = sorted(list(set(df.CountryName)))
hist_ips_df = pd.read_csv(path_to_ips_file,
                              parse_dates=['Date'],
                              encoding="ISO-8859-1",
                              dtype={"RegionName": str},
                              error_bad_lines=True)
hist_ips_df = hist_ips_df[hist_ips_df.CountryName.isin(list_countries)]
hist_ips_df.to_csv("data/2020-09-30_historical_ip_new.csv" , index = False)

In [31]:
%%time
preds_df = predict_df("2020-08-01", "2020-08-31", path_to_ips_file="data/2020-09-30_historical_ip_new.csv", verbose=True)


Predicting for Aruba__nan
2020-08-01: 58.832013097676466
2020-08-02: 71.32297323510508
2020-08-03: 78.9884411385919
2020-08-04: 90.24123842546683
2020-08-05: 87.8117472014118
2020-08-06: 98.7600349484055
2020-08-07: 131.043712129859
2020-08-08: 147.02330387430604
2020-08-09: 158.9529114875828
2020-08-10: 169.9426640501474
2020-08-11: 177.51782587473804
2020-08-12: 193.4843233415486
2020-08-13: 216.8238449408644
2020-08-14: 234.0413552003931
2020-08-15: 248.5614891807204
2020-08-16: 261.9688949069497
2020-08-17: 275.3003157177194
2020-08-18: 293.29289528071473
2020-08-19: 314.0776214183259
2020-08-20: 332.4304943890407
2020-08-21: 349.2338921371767
2020-08-22: 365.50191374821793
2020-08-23: 382.5349053058303
2020-08-24: 407.89251745148283
2020-08-25: 430.0842793419013
2020-08-26: 450.5729646715425
2020-08-27: 466.0014796361783
2020-08-28: 484.648520643598
2020-08-29: 515.3546084163565
2020-08-30: 541.9068844107129
2020-08-31: 566.2680616270064

Predicting for Afghanistan__nan
2020-08-0

2020-08-21: 7910.84537081261
2020-08-22: 7860.659200471844
2020-08-23: 7807.965424691464
2020-08-24: 7994.498827564898
2020-08-25: 8369.232526121434
2020-08-26: 8630.390825116065
2020-08-27: 8742.663609313939
2020-08-28: 8786.624652833447
2020-08-29: 8860.458626042933
2020-08-30: 9066.839840404056
2020-08-31: 9351.180796573675

Predicting for Azerbaijan__nan
2020-08-01: 2143.905508864326
2020-08-02: 2777.115434179791
2020-08-03: 3002.283203813526
2020-08-04: 2959.1106050606804
2020-08-05: 2639.3525037681984
2020-08-06: 1674.8565956871844
2020-08-07: 2515.0067092880695
2020-08-08: 2970.024579963173
2020-08-09: 3126.5901473337626
2020-08-10: 3100.8489962529766
2020-08-11: 2834.0265596383724
2020-08-12: 2564.8487562942364
2020-08-13: 2961.312889098826
2020-08-14: 3256.72709783517
2020-08-15: 3377.044136433693
2020-08-16: 3363.6585710101017
2020-08-17: 3234.8695147372355
2020-08-18: 3208.235017564942
2020-08-19: 3439.482666855574
2020-08-20: 3639.9084674373753
2020-08-21: 3740.560764483733

2020-08-12: 1175.4373465056506
2020-08-13: 1408.9578552954586
2020-08-14: 1502.5692836350077
2020-08-15: 1525.8530719076202
2020-08-16: 1491.9720669482945
2020-08-17: 1438.3819367637952
2020-08-18: 1489.537375319962
2020-08-19: 1620.9651836969206
2020-08-20: 1695.6945168657362
2020-08-21: 1723.7103507499196
2020-08-22: 1719.9150119042924
2020-08-23: 1720.9199044274983
2020-08-24: 1778.277004191289
2020-08-25: 1867.4455935629344
2020-08-26: 1930.6044309928125
2020-08-27: 1957.2832946137587
2020-08-28: 1975.0815736919785
2020-08-29: 2003.2467950824603
2020-08-30: 2061.037070764019
2020-08-31: 2132.0587311890354

Predicting for Belarus__nan
2020-08-01: 1204.9756095394246
2020-08-02: 1465.4856591021608
2020-08-03: 1541.6280481334315
2020-08-04: 1617.1840586861654
2020-08-05: 1378.408580939371
2020-08-06: 920.1021009700008
2020-08-07: 1392.7190497155634
2020-08-08: 1601.8250014185369
2020-08-09: 1680.465067242601
2020-08-10: 1697.3212982907103
2020-08-11: 1543.4586636759345
2020-08-12: 1426

2020-08-09: 231.62158240389508
2020-08-10: 219.0780145826726
2020-08-11: 231.0635236390047
2020-08-12: 266.63492628815874
2020-08-13: 324.9463260095422
2020-08-14: 350.0876429416763
2020-08-15: 332.8528332560628
2020-08-16: 333.2197209456309
2020-08-17: 350.5754582667661
2020-08-18: 383.32698713988026
2020-08-19: 422.1749993602521
2020-08-20: 441.73883734818236
2020-08-21: 444.2501956055553
2020-08-22: 454.40558640575694
2020-08-23: 475.3475388524662
2020-08-24: 505.04586861756536
2020-08-25: 535.4272964111753
2020-08-26: 555.4542009113054
2020-08-27: 568.4348070328069
2020-08-28: 585.445012060351
2020-08-29: 609.0219550333837
2020-08-30: 637.2827415238758
2020-08-31: 664.9274295961122

Predicting for Central African Republic__nan
2020-08-01: 52.42141290904502
2020-08-02: 67.10802689864575
2020-08-03: 71.15272221074937
2020-08-04: 74.75896665844084
2020-08-05: 79.93724114044045
2020-08-06: 93.88638611768256
2020-08-07: 124.20218969464617
2020-08-08: 140.1179120513649
2020-08-09: 149.33

2020-08-25: 523.2783448917694
2020-08-26: 551.7702035883752
2020-08-27: 569.7526142879846
2020-08-28: 589.475559964872
2020-08-29: 611.5719444948255
2020-08-30: 638.5091102373596
2020-08-31: 670.7195773722983

Predicting for Congo__nan
2020-08-01: 57.568783553915985
2020-08-02: 89.78866142637972
2020-08-03: 126.49334954427641
2020-08-04: 89.61131477153927
2020-08-05: 88.23144833664318
2020-08-06: 102.93427003307653
2020-08-07: 138.27296852834803
2020-08-08: 167.3841602083289
2020-08-09: 185.26331920398542
2020-08-10: 177.8871612315016
2020-08-11: 183.70900617093798
2020-08-12: 201.80632433321946
2020-08-13: 228.83597200312005
2020-08-14: 252.90598715754686
2020-08-15: 267.93873995115285
2020-08-16: 273.75867603726766
2020-08-17: 285.6732487452306
2020-08-18: 310.8757781765562
2020-08-19: 335.9210360500456
2020-08-20: 358.24568283941187
2020-08-21: 380.9862086413999
2020-08-22: 395.95476136544494
2020-08-23: 424.1644567221687
2020-08-24: 451.0923626668222
2020-08-25: 477.2421086074804
2

...

2020-08-02: 3659.0241770514567
2020-08-03: 3726.6344667757994
2020-08-04: 3877.3668413089254
2020-08-05: 2690.693659528012
2020-08-06: 2105.5144532979266
2020-08-07: 3463.4904418450815
2020-08-08: 3822.5362440658105
2020-08-09: 3930.757328878009
2020-08-10: 3843.721536218235
2020-08-11: 3285.128618979386
2020-08-12: 3212.3979195946135
2020-08-13: 3840.6432030915166
2020-08-14: 4125.520296082497
2020-08-15: 4217.131008537805
2020-08-16: 4133.705879799106
2020-08-17: 3913.990260737907
2020-08-18: 3990.01311816484
2020-08-19: 4334.269820096103
2020-08-20: 4545.91158649552
2020-08-21: 4629.037733610194
2020-08-22: 4603.9729138582625
2020-08-23: 4566.868922175874
2020-08-24: 4682.893095978566
2020-08-25: 4909.149387046618
2020-08-26: 5076.517524032368
2020-08-27: 5161.058107319829
2020-08-28: 5193.483253652569
2020-08-29: 5239.426735145435
2020-08-30: 5367.889580632669
2020-08-31: 5544.061745395258

Predicting for United States__North Dakota
2020-08-01: 1170.7487697027623
2020-08-02: 1396.3

2020-08-17: 9876.138483711084
2020-08-18: 9650.002779866501
2020-08-19: 10259.138451244593
2020-08-20: 10764.533378180453
2020-08-21: 11043.858706324838
2020-08-22: 11164.74553587387
2020-08-23: 11101.848149417863
2020-08-24: 11174.833604444957
2020-08-25: 11591.079598454988
2020-08-26: 11980.338420371054
2020-08-27: 12241.387958803089
2020-08-28: 12396.848542143682
2020-08-29: 12487.389867092004
2020-08-30: 12680.522122823253
2020-08-31: 13025.026950396385

Predicting for United States__Oklahoma
2020-08-01: 2606.266689582806
2020-08-02: 3201.5071424935195
2020-08-03: 3511.182594975134
2020-08-04: 3505.6398529537696
2020-08-05: 3006.5437536934196
2020-08-06: 1951.9180468501909
2020-08-07: 2963.4923502799584
2020-08-08: 3446.6208630150536
2020-08-09: 3647.2278546283983
2020-08-10: 3617.7370610508983
2020-08-11: 3266.014320591125
2020-08-12: 2978.5160556194787
2020-08-13: 3449.901175838974
2020-08-14: 3778.2126165666396
2020-08-15: 3922.5594488903916
2020-08-16: 3900.703112376714
2020-08

2020-08-01: 3333.245367887409
2020-08-02: 3947.972010817269
2020-08-03: 3488.84327881831
2020-08-04: 3215.198603101952
2020-08-05: 2459.1792467956466
2020-08-06: 2005.2242633104634
2020-08-07: 3407.8999344497797
2020-08-08: 3824.2701617985213
2020-08-09: 3659.6966608450434
2020-08-10: 3427.81078573791
2020-08-11: 3044.469291169825
2020-08-12: 3058.290286788274
2020-08-13: 3722.354999782284
2020-08-14: 3989.9212119295016
2020-08-15: 3944.316053658835
2020-08-16: 3808.945022571046
2020-08-17: 3666.299044442675
2020-08-18: 3793.460778590616
2020-08-19: 4154.291399024711
2020-08-20: 4337.470439005201
2020-08-21: 4351.156124202112
2020-08-22: 4305.972183222518
2020-08-23: 4304.3898335452595
2020-08-24: 4447.497727937205
2020-08-25: 4677.724438925723
2020-08-26: 4821.1575952582525
2020-08-27: 4869.250940428613
2020-08-28: 4892.408598680961
2020-08-29: 4954.29397930594
2020-08-30: 5094.939060652214
2020-08-31: 5268.628570490043

Predicting for United States__Virginia
2020-08-01: 1918.14010302

2020-08-17: 471.7830316976523
2020-08-18: 491.6491641895873
2020-08-19: 524.9603942388293
2020-08-20: 550.0768307231419
2020-08-21: 567.610843085466
2020-08-22: 587.60249149031
2020-08-23: 619.5081495550946
2020-08-24: 646.837824545383
2020-08-25: 677.5508868411483
2020-08-26: 704.5716234839787
2020-08-27: 728.4643368444756
2020-08-28: 755.042969779906
2020-08-29: 786.2892344270618
2020-08-30: 817.0886307163855
2020-08-31: 848.7066869513257

Predicting for Venezuela__nan
2020-08-01: 318.2055101062801
2020-08-02: 342.90450384636677
2020-08-03: 273.9616160435894
2020-08-04: 488.20098851203176
2020-08-05: 583.842675187388
2020-08-06: 324.4420121481896
2020-08-07: 422.7282950422875
2020-08-08: 448.2073888778224
2020-08-09: 463.24210003947354
2020-08-10: 574.5364772032209
2020-08-11: 595.3838506038588
2020-08-12: 516.0928778692054
2020-08-13: 555.3427394840324
2020-08-14: 583.1672346692476
2020-08-15: 618.359715952358
2020-08-16: 679.7089393063504
2020-08-17: 692.4838651362011
2020-08-18: 6

Wall time: 1min 6s


In [32]:
# Check the predictions
preds_df.head()

,CountryName,RegionName,Date,PredictedDailyNewCases
213,Aruba,NaN,2020-08-01,58.832013
214,Aruba,NaN,2020-08-02,71.322973
215,Aruba,NaN,2020-08-03,78.988441
216,Aruba,NaN,2020-08-04,90.241238
217,Aruba,NaN,2020-08-05,87.811747


# Validation
This is how the predictor is going to be called during the competition.  
!!! PLEASE DO NOT CHANGE THE API !!!

In [37]:
!python predict.py -s 2020-08-01 -e 2020-08-04 -ip data/2020-09-30_historical_ip_new.csv -o predictions/2020-08-01_2020-08-04.csv

Generating predictions from 2020-08-01 to 2020-08-04...
Saved predictions to predictions/2020-08-01_2020-08-04.csv
Done!


In [45]:
!head predictions/2020-08-01_2020-08-04.csv

'head' 不是内部或外部命令，也不是可运行的程序
或批处理文件。


# Test cases
We can generate a prediction file. Let's validate a few cases...

In [40]:
import os
from covid_xprize.validation.predictor_validation import validate_submission

def validate(start_date, end_date, ip_file, output_file):
    # First, delete any potential old file
    try:
        os.remove(output_file)
    except OSError:
        pass
    
    # Then generate the prediction, calling the official API
    !python predict.py -s {start_date} -e {end_date} -ip {ip_file} -o {output_file}
    
    # And validate it
    errors = validate_submission(start_date, end_date, ip_file, output_file)
    if errors:
        for error in errors:
            print(error)
    else:
        print("All good!")

ModuleNotFoundError: No module named 'covid_xprize'

## 4 days, no gap
- All countries and regions
- Official number of cases is known up to start_date
- Intervention Plans are the official ones

In [ ]:
validate(start_date="2020-08-01",
         end_date="2020-08-04",
         ip_file="../../../validation/data/2020-09-30_historical_ip.csv",
         output_file="predictions/val_4_days.csv")

## 1 month in the future
- 2 countries only
- there's a gap between date of last known number of cases and start_date
- For future dates, Intervention Plans contains scenarios for which predictions are requested to answer the question: what will happen if we apply these plans?

In [ ]:
%%time
validate(start_date="2021-01-01",
         end_date="2021-01-31",
         ip_file="../../../validation/data/future_ip.csv",
         output_file="predictions/val_1_month_future.csv")

## 180 days, from a future date, all countries and regions
- Prediction start date is 1 week from now. (i.e. assuming submission date is 1 week from now)  
- Prediction end date is 6 months after start date.  
- Prediction is requested for all available countries and regions.  
- Intervention plan scenario: freeze last known intervention plans for each country and region.  

As the number of cases is not known yet between today and start date, but the model relies on them, the model has to predict them in order to use them.  
This test is the most demanding test. It should take less than 1 hour to generate the prediction file.

### Generate the scenario

In [ ]:
from datetime import datetime, timedelta

start_date = datetime.now() + timedelta(days=7)
start_date_str = start_date.strftime('%Y-%m-%d')
end_date = start_date + timedelta(days=180)
end_date_str = end_date.strftime('%Y-%m-%d')
print(f"Start date: {start_date_str}")
print(f"End date: {end_date_str}")

In [ ]:
from covid_xprize.validation.scenario_generator import get_raw_data, generate_scenario, NPI_COLUMNS
DATA_FILE = 'data/OxCGRT_latest.csv'
latest_df = get_raw_data(DATA_FILE, latest=True)
scenario_df = generate_scenario(start_date_str, end_date_str, latest_df, countries=None, scenario="Freeze")
scenario_file = "predictions/180_days_future_scenario.csv"
scenario_df.to_csv(scenario_file, index=False)
print(f"Saved scenario to {scenario_file}")

### Check it

In [ ]:
%%time
validate(start_date=start_date_str,
         end_date=end_date_str,
         ip_file=scenario_file,
         output_file="predictions/val_6_month_future.csv")